In [2]:
import ee

In [4]:
ee.Authenticate()

Enter verification code:  4/1AdQt8qhITDGG1wtjv4NRBE1GQLrHyW-r1nqyelLGkgu9cVT20XTT1EqO424



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
import importlib
from flood_detection import modis
importlib.reload(modis)
from flood_detection.utils import misc
import geemap
from pathlib import Path
import rasterio
from pathlib import Path
import multiprocessing
from functools import partial

In [18]:
import HelperMethods.ExportChipsHelper as ExportHelper
importlib.reload(ExportHelper)

<module 'HelperMethods.ExportChipsHelper' from '/home/jovyan/work/HelperMethods/ExportChipsHelper.py'>

In [6]:
exportPath = Path('/Data/Raster/ModisHistoricalGFD/')

In [14]:
def downloadChip(year, chip, chipID, chip_i, fileList):
    name = chipID + "_" + str(chip_i) + "_" + str(year)
    if name in fileList:
        return
    
    geom = chip

    filename = exportPath/(name + '.tif')

    flood_map = modis.dfo(geom, str(year)+"-05-01", str(year)+"-11-30", "standard", '3Day', True)

    flood_map_slope_mask = misc.apply_slope_mask(flood_map, thresh=5)

    perm_water = misc.get_jrc_perm(geom)

    dfo_final = ee.Image(flood_map_slope_mask).addBands(perm_water)

    geemap.ee_export_image(dfo_final,\
                       filename,\
                       scale=250,\
                       crs='EPSG:4326',\
                       region=geom,\
                       file_per_band=False)

In [16]:
fileList = ExportHelper.getFilesNamesList(exportPath)

chipIDs = ['09', '0a', '0f', '10'] #, 
chips = ee.FeatureCollection("projects/ee-bobgiezi/assets/ShapeFiles/OverlapGrid")
for chipID in chipIDs:
    print('---- ' + str(chipID) + ' ----')
    fullChip = chips.filter(ee.Filter.eq('system:index', '000000000000000000' + chipID)).first()
    finerGrid = ExportHelper.createFineGrid(fullChip)
    listSize = finerGrid.size().getInfo()
    finerGridList = finerGrid.toList(listSize)
    for chip_i in range(listSize):
        print(f"processing chip {chip_i+1}/{listSize}")
        chip = ee.Feature(finerGridList.get(chip_i)).geometry()
        with multiprocessing.Pool(5) as pool:
            pool.map(partial(downloadChip, chip=chip, chipID=chipID, chip_i=chip_i, fileList=fileList), range(2001, 2022))

---- 09 ----
processing chip 1/64
processing chip 2/64
processing chip 3/64
processing chip 4/64
processing chip 5/64
processing chip 6/64
processing chip 7/64
processing chip 8/64
processing chip 9/64
processing chip 10/64
processing chip 11/64
processing chip 12/64
processing chip 13/64
processing chip 14/64
processing chip 15/64
processing chip 16/64
processing chip 17/64
processing chip 18/64
processing chip 19/64
processing chip 20/64
processing chip 21/64
processing chip 22/64
processing chip 23/64
processing chip 24/64
processing chip 25/64
processing chip 26/64
processing chip 27/64
processing chip 28/64
processing chip 29/64
processing chip 30/64
processing chip 31/64
processing chip 32/64
processing chip 33/64
processing chip 34/64
processing chip 35/64
processing chip 36/64
processing chip 37/64
processing chip 38/64
processing chip 39/64
processing chip 40/64
processing chip 41/64
processing chip 42/64
processing chip 43/64
processing chip 44/64
processing chip 45/64
proces

In [19]:
fileList = ExportHelper.getFilesList(exportPath)
for file in fileList:
    with rasterio.open(file,'r+') as dst:
        dst.descriptions = tuple(['flooded', 'duration', 'clearViews', 'clearPerc', 'maxExtent', 'permanentWater'])